In [ ]:
from PIL import Image as Img
from PIL import ImageTk

class ageDataset(Dataset): 
    def __init__(self, image,label, train=True, transform=None): 
        self.transform = transform 
        self.img_list = image
        self.label_list=label
    def __len__(self): 
        return len(self.img_list) 
    def __getitem__(self, idx): 
        label = self.label_list[idx] 
        img = Img.fromarray(np.uint8(self.img_list[idx])).convert('RGB')

        if self.transform is not None: 
            img = self.transform(img) 
            
        return img, label 

In [ ]:
path = " " #이미지 패스
# ex)'/content/drive/MyDrive/Deepruning_framework/datasets/3.jpg'
person = glob.glob(path)
img = cv2.imread(path) 
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) #pyplot에서 보기위해서 BGR을 RGB로 변환

plt.imshow(img)
plt.show()

In [ ]:
num = 0
i = 0
non_recognition_idx = []
face_image = []

image = fr.load_image_file(person[0]) #넘파이로 이미지 불러들임임
encodings = fr.face_encodings(image)

if len(encodings) > 0:
    biden_encoding = encodings[0]
    top,right,bottom,left = fr.face_locations(image)[0]
    face_image1 = image[top:bottom,left:right]
    face_image.append(face_image1)
else:
    non_recognition_idx.append(i)
    num +=1
i = i+1

if num == 1:
  print("얼굴이 잘 인식되자 않습니다 다른환경에서 사진찍거나 다른사진을 사용해보세요")

else:
  plt.imshow(face_image[0])
  plt.show()

In [ ]:
test_transform = torchvision.transforms.Compose([
                    transforms.Resize([CFG['IMG_SIZE'], CFG['IMG_SIZE']]),#각 이미지 같은 크기로 resize
                    transforms.ToTensor(),#이미지를 텐서로 변환
                    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))#평균과 표준편차를 0.5로 정규화
                    ])

In [ ]:
test_dataset= ageDataset(image=face_image,label=[0], train=False, transform=test_transform)  #validation custom dataset 구축
test_loader = DataLoader(test_dataset, batch_size = 64, num_workers=0) #dataloadet 대입 

In [ ]:
model1 = torchvision.models.resnet18(torchvision.models.ResNet18_Weights)

In [ ]:
check_point=torch.load('./best_model.pth')
model=model1
model=model.to(device)
model.load_state_dict(check_point)

In [ ]:
with torch.no_grad(): #파라미터 업데이트 안하기 때문에 no_grad 사용
            for img, label in tqdm(iter(test_loader)):
                img, label = img.to(device), label.to(device)

                logit = model(img)
                pred = logit.argmax(dim=1, keepdim=True)
                print(logit)
                print(pred)

In [ ]:
pred = pred.tolist()
logit = logit.tolist()
print(logit)
print(pred)

In [ ]:
if(pred[0][0]==0):
  print("{:.2f}확률로 미성년자입니다.".format(logit[0][pred[0][0]]))
if(pred[0][0]==1):
  print("{:.2f}확률로 청년층입니다.".format(logit[0][pred[0][0]]))
if(pred[0][0]==2):
  print("{:.2f}확률로 중년층입니다.".format(logit[0][pred[0][0]]))
if(pred[0][0]==3):
  print("{:.2f}확률로 노년층자입니다.".format(logit[0][pred[0][0]]))